# Data Poisoning

read article: https://www.promptfoo.dev/blog/rag-poisoning/  for more information

In [8]:
#initial setup
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils import OPENAI_API_KEY

In [9]:
"""
For this worksheet we have created a wrapper around chromadb to simplify the worksheet
TODO: take a look at the source to see how it is implemented
"""

from typing import List, Tuple
import openai

GPT_4 = 'gpt-4-turbo' # 128,000 tokens


from src.chroma_db import VectorCollection, OpenAIEmbeddingModel, get_chromadb_client, remove_collection
from src.openai import get_response

SCHEMA_NAME = "data_poisoning_embedding"
chroma_client = get_chromadb_client(SCHEMA_NAME)
client_openai = openai.OpenAI(
    api_key=OPENAI_API_KEY
)

def load_example_text()->str:
    with open("example_document.txt", "r") as file:
        txt = file.read()
    return txt


FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
"""
TODO: run cell -> to populate vector store with the chunks created in the cell above
"""

COLLECTION_USER_EXPERIMENTAL_NAME = "poisoning_experimental"


def drop_and_populate_collection(collection: VectorCollection, chunks: List[str], chroma_client)->None:
    remove_collection(chroma_client, COLLECTION_USER_EXPERIMENTAL_NAME)
    print(f"going to embed {len(chunks)} chunks")
    for index, chunk in enumerate(chunks):
        print(f"adding chunk {index}")
        collection.add_item(chunk, f"id_{index}")

collection_user_experimental = VectorCollection(COLLECTION_USER_EXPERIMENTAL_NAME,
                                                chroma_client,
                                                OPENAI_API_KEY,
                                                embedding_model=OpenAIEmbeddingModel.ADA_002)



In [ ]:
documents = [
    "a pragmatic guide to dealing with data - a comprehensive overview of data management",
    "data systems and algorithms - a comprehensive overview of data management",
    "data processing in the age of AI- a comprehensive overview of data management",
    "data for dummies - a comprehensive overview of data management",
    "integration of data systems and IOT - a comprehensive overview of data management",
    "Important Urgent Important Urgent Data book - a book written by rushil daya",
    "dealing with big data - a comprehensive overview of data management"
]

drop_and_populate_collection(collection_user_experimental, documents, chroma_client)

In [ ]:
search_term = "I am looking for a book on data"

closest_items = collection_user_experimental.similar_items(search_term, n_results=1)

for item in closest_items:
    print(f"distance_to_search_term: {item.distance}, id: {item.id} ,item: {item.text}")


In [ ]:
prompt_with_context = f"""
answer the users question giving priority to the context provided in the following list of documents:
{[str(index)+': '+item.text+'\n' for index, item in enumerate(closest_items)]}
User question: {search_term}
"""

In [ ]:
get_response(client_openai, prompt_with_context, GPT_4)

In [ ]:
#TODO add a filter on the list to try and remove the poisoned document making it as robust as possible
# add more documents examples of poisoned documents to test your filter